In [1]:
import numpy as np

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)  # vdot handles complex conjugation automatically

# Define the qubit states as complex numpy arrays
psi = (1 / np.sqrt(6)) * np.array([2, 1 + 1j])
phi = (1 / np.sqrt(10)) * np.array([1 - 1j, 3])

# Compute the inner product
inner_prod = inner_product(psi, phi)

# Display the result
print("Inner product:", inner_prod)

Inner product: (0.6454972243679029-0.6454972243679029j)


In [3]:
import numpy as np

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)  # vdot handles complex conjugation automatically

def encode_to_qubit(arr):
    """Encodes a real array of size 4 into a qubit state using amplitude encoding."""
    assert len(arr) == 4, "Array must have exactly 4 elements"
    complex_vec = np.array([arr[0] + 1j * arr[1], arr[2] + 1j * arr[3]])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

# Define two real arrays of size 4
real_array1 = np.array([2, 1, 3, -1])
real_array2 = np.array([1, -1, 4, 2])

# Encode them into qubit states
psi = encode_to_qubit(real_array1)
phi = encode_to_qubit(real_array2)

# Compute the inner product
inner_prod = inner_product(psi, phi)

# Compute the magnitude of the inner product
inner_magnitude = np.abs(inner_prod)

# Display the results
print("Inner product:", inner_prod)
print("Magnitude of inner product:", inner_magnitude)


Inner product: (0.6055300708194982+0.38533731779422614j)
Magnitude of inner product: 0.7177405625652732


In [5]:
import json
import os

# Define the file to store embeddings
EMBEDDINGS_FILE = "embeddings.json"

def load_embeddings():
    """Loads stored embeddings from a JSON file if it exists."""
    if os.path.exists(EMBEDDINGS_FILE):
        with open(EMBEDDINGS_FILE, "r") as f:
            return json.load(f)
    return {}

def get_embeddings(word, stored_embeddings):
    """Retrieves embedding from cache or fetches it from OpenAI if not cached."""
    if word in stored_embeddings:
        print(f"Loaded cached embedding for '{word}'")
        return stored_embeddings[word]

def encode_to_qubit(arr):
    """Encodes a real array of any even size into a qubit state using amplitude encoding."""
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

words = ["cat", "dog", "car"]

# Load existing embeddings
stored_embeddings = load_embeddings()

# Retrieve embeddings (from file or OpenAI)
embeddings = {word: get_embeddings(word, stored_embeddings) for word in words}

# Extract individual vectors correctly
cat_vec = embeddings["cat"]
dog_vec = embeddings["dog"]
car_vec = embeddings["car"]

# Encode them into qubit states
cat_state = encode_to_qubit(cat_vec)
dog_state = encode_to_qubit(dog_vec)
car_state = encode_to_qubit(car_vec)

# Compute the inner product
cat_dog_inner_prod = inner_product(cat_state, dog_state)
cat_car_inner_prod = inner_product(cat_state, car_state)

# Compute the magnitude of the inner product
cat_dog_magnitude = np.abs(cat_dog_inner_prod)
cat_car_magnitude = np.abs(cat_car_inner_prod)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod)
print("cat/car Inner product:", cat_car_inner_prod)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude)
print("cat/car Magnitude of inner product:", cat_car_magnitude)


Loaded cached embedding for 'cat'
Loaded cached embedding for 'dog'
Loaded cached embedding for 'car'
cat/dog Inner product: (0.862957645542653-0.016694274937911695j)
cat/car Inner product: (0.845207835908355-0.010998340229826991j)
cat/dog Magnitude of inner product: 0.8631191092869059
cat/car Magnitude of inner product: 0.8452793913072149


In [19]:
from scipy.linalg import expm

def complex_amplitude_encoding(vector, t=np.pi/2, steps=100):
    vector = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(vector)
    
    if norm == 0:
        raise ValueError("Cannot encode a zero vector.")
    
    target_state = vector / norm
    dim = len(target_state)

    # Initial Hamiltonian (transverse field) with |+> = (|0> + |1>) / sqrt(2)
    H0 = -np.ones((dim, dim), dtype=complex) / dim  # Approximate uniform superposition state
    
    # Final Hamiltonian (encoding target state)
    v = np.zeros(dim, dtype=complex)
    v[:dim] = target_state  
    H1 = np.outer(v, v.conj())
    
    # Perform adiabatic evolution
    psi = np.ones(dim, dtype=complex) / np.sqrt(dim)  # Start in |+>
    dt = t / steps
    
    for s in np.linspace(0, 1, steps):
        H = (1 - s) * H0 + s * H1  # Interpolated Hamiltonian
        U = expm(-1j * dt * H)  # Small evolution step
        psi = U @ psi  # Apply evolution step
    
    return psi

# Encode them into qubit states
cat_state2 = complex_amplitude_encoding(cat_vec)
dog_state2 = complex_amplitude_encoding(dog_vec)
car_state2 = complex_amplitude_encoding(car_vec)

cats_inner_prod = inner_product(cat_state, cat_state2)
cats_magnitude = np.abs(cats_inner_prod)

print("cat/cat Magnitude of inner product:", cats_magnitude)

# Compute the inner product
cat_dog_inner_prod2 = inner_product(cat_state2, dog_state2)
cat_car_inner_prod2 = inner_product(cat_state2, car_state2)

# Compute the magnitude of the inner product
cat_dog_magnitude2 = np.abs(cat_dog_inner_prod2)
cat_car_magnitude2 = np.abs(cat_car_inner_prod2)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod2)
print("cat/car Inner product:", cat_car_inner_prod2)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude2)
print("cat/car Magnitude of inner product:", cat_car_magnitude2)

cat/cat Magnitude of inner product: 0.03034050285073909
cat/dog Inner product: (0.9998804665899756-6.763244320667772e-05j)
cat/car Inner product: (0.9999001158540667+5.260402786194662e-05j)
cat/dog Magnitude of inner product: 0.9998804688773227
cat/car Magnitude of inner product: 0.9999001172377968


In [21]:
def complex_amplitude_encoding_vector(vector, t=np.pi/2):
    """
    Encodes a real-valued vector of arbitrary even length into a quantum state using Hamiltonian evolution.
    """
    assert len(vector) % 2 == 0, "Vector length must be even"
    
    # Convert real vector into complex amplitudes
    vector = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(vector)
    
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    
    target_state = vector / norm
    dim = len(target_state)
    
    # Define Hamiltonian based on the target state
    H = np.outer(target_state, target_state.conj())  # Projector onto the target state
    
    # Compute the unitary evolution operator U = exp(-i * H * t)
    U = expm(-1j * t * H)
    
    # Apply evolution to the initial |+> state
    initial_state = np.ones(dim, dtype=complex) / np.sqrt(dim)  # Start in |+>
    evolved_state = U @ initial_state
    
    return evolved_state

# Encode them into qubit states
cat_state3 = complex_amplitude_encoding_vector(cat_vec)

cats_inner_prod12 = inner_product(cat_state, cat_state2)
cats_inner_prod13 = inner_product(cat_state, cat_state2)
cats_inner_prod23 = inner_product(cat_state2, cat_state3)

cats_magnitude12 = np.abs(cats_inner_prod12)
cats_magnitude13 = np.abs(cats_inner_prod13)
cats_magnitude23 = np.abs(cats_inner_prod23)

print("cat1/cat2 Magnitude of inner product:", cats_magnitude12)
print("cat1/cat3 Magnitude of inner product:", cats_magnitude13)
print("cat2/cat3 Magnitude of inner product:", cats_magnitude23)

cat1/cat2 Magnitude of inner product: 0.03034050285073909
cat1/cat3 Magnitude of inner product: 0.03034050285073909
cat2/cat3 Magnitude of inner product: 0.9996960440707399


In [27]:
import numpy as np
from scipy.linalg import expm

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)  # vdot handles complex conjugation automatically

def encode_to_qubit(arr):
    """Encodes a real array of any even size into a qubit state using amplitude encoding."""
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def controlled_rotation(theta, dim):
    """
    Constructs a controlled rotation matrix R_y(theta) for an n-qubit system.
    """
    I = np.eye(dim//2)
    Ry = np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2), np.cos(theta/2)]
    ], dtype=complex)
    return np.kron(I, Ry)

def state_preparation(vector):
    """
    Implements amplitude encoding for an n-qubit system using controlled rotations.
    """
    assert len(vector) % 2 == 0, "Vector length must be even."
    dim = len(vector) // 2
    norm = np.linalg.norm(vector)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    
    vector = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    vector /= norm  # Normalize the vector
    
    # Compute angles for rotations
    angles = [2 * np.arccos(np.sqrt(np.sum(np.abs(vector[:2**(i+1)])**2))) for i in range(int(np.log2(dim)))]
    
    # Initial state |+...+>
    psi = np.ones(dim, dtype=complex) / np.sqrt(dim)

    # Start with |0...0>
    #psi = np.zeros(dim, dtype=complex)
    #psi[0] = 1.0  # Initial state |000...0>
    
    # Apply controlled rotations iteratively
    for theta in angles:
        U = controlled_rotation(theta, dim)
        psi = U @ psi
    
    return psi

cat_state4 = state_preparation(cat_vec)

cats_inner_prod14 = inner_product(cat_state, cat_state4)
cats_magnitude14 = np.abs(cats_inner_prod14)

cats_inner_prod34 = inner_product(cat_state3, cat_state4)
cats_magnitude34 = np.abs(cats_inner_prod34)

print("cat1/cat4 Magnitude of inner product:", cats_magnitude14)
print("cat3/cat4 Magnitude of inner product:", cats_magnitude34)


cat1/cat4 Magnitude of inner product: 0.0356657962458934
cat3/cat4 Magnitude of inner product: 0.0436161746619703


In [29]:
import numpy as np
from scipy.linalg import expm

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)  # vdot handles complex conjugation automatically

def encode_to_qubit(arr):
    """Encodes a real array of any even size into a qubit state using amplitude encoding."""
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def quantum_state_preparation(vector):
    """
    Simulates amplitude encoding on real quantum hardware using an optimized state preparation approach.
    """
    assert len(vector) % 2 == 0, "Vector length must be even."

    # Convert real vector into complex amplitudes
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])

    # Normalize the vector to ensure it's a valid quantum state
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    dim = len(target_state)
    num_qubits = int(np.log2(dim))  # Number of qubits required

    # Initialize the state as |0...0>
    psi = np.zeros(dim, dtype=complex)
    psi[0] = 1.0  # Start in |000...0>

    # Construct the exact unitary transformation to map |0...0> to target_state
    U = np.eye(dim, dtype=complex)
    for i in range(dim):
        U[:, i] = target_state  # Replace each column with the correct amplitudes

    # Apply the transformation
    psi = U @ psi

    return psi

# Example vector for encoding
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Encode state using both methods
encoded_state_direct = encode_to_qubit(vector)
encoded_state_simulated = quantum_state_preparation(vector)

# Verify if both methods result in the same state
comparison_inner_prod = inner_product(encoded_state_direct, encoded_state_simulated)
comparison_magnitude = np.abs(comparison_inner_prod)

# Display the results
print("Encoded Quantum State (Direct Method):", encoded_state_direct)
print("Encoded Quantum State (Hardware-Simulated):", encoded_state_simulated)
print("Inner Product Magnitude Between Methods:", comparison_magnitude)

# Check if they are approximately the same
if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states.")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.31976474+0.31976474j 0.31976474+0.31976474j 0.15988237+0.47964711j
 0.06395295+0.57557653j]
Encoded Quantum State (Hardware-Simulated): [0.31976474+0.31976474j 0.31976474+0.31976474j 0.15988237+0.47964711j
 0.06395295+0.57557653j]
Inner Product Magnitude Between Methods: 1.0
The two methods produce equivalent quantum states.


In [36]:
import numpy as np
from scipy.linalg import expm

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)  # vdot handles complex conjugation automatically

def encode_to_qubit(arr):
    """Encodes a real array of any even size into a qubit state using amplitude encoding."""
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Constructs an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2), np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Constructs an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def apply_controlled_rotations(state, amplitudes, phases):
    """
    Applies a structured sequence of controlled R_y and R_z rotations
    following a binary-tree pattern to match the target state.
    """
    dim = len(state)
    num_levels = int(np.log2(dim))  # Number of qubits

    # Apply controlled rotations
    for level in range(num_levels):
        step = 2**(level + 1)
        for i in range(0, dim, step):  # Step through pairs
            j = i + step // 2
            if j < dim:
                # Compute correct rotation angles
                theta = amplitudes[j]
                phi = phases[j]
                
                Ry = rotation_matrix_y(theta)
                Rz = rotation_matrix_z(phi)
                
                # Apply R_y first, then R_z
                substate = np.array([state[i], state[j]])
                rotated_substate = Rz @ Ry @ substate
                state[i], state[j] = rotated_substate[0], rotated_substate[1]
    
    return state

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using real quantum hardware principles.

    - Uses only unitary operations (controlled rotations and phase shifts)
    - Starts from |0...0> and gradually evolves the state
    """
    assert len(vector) % 2 == 0, "Vector length must be even."

    # Convert real vector into complex amplitudes
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])

    # Normalize the vector to ensure it's a valid quantum state
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    dim = len(target_state)
    
    # Start with |0...0>
    psi = np.zeros(dim, dtype=complex)
    psi[0] = 1.0  # Initial state |000...0>

    # Compute angles for rotations using arccos
    amplitudes = [2 * np.arccos(np.abs(target_state[i])) for i in range(dim)]
    phases = [np.angle(target_state[i]) for i in range(dim)]  # Extract phase

    # Apply controlled rotations iteratively
    psi = apply_controlled_rotations(psi, amplitudes, phases)

    return psi

# Example vector for encoding
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Encode state using both methods
encoded_state_direct = encode_to_qubit(vector)
encoded_state_hardware = amplitude_encoding_real_hardware(vector)

# Verify if both methods result in the same state
comparison_inner_prod = inner_product(encoded_state_direct, encoded_state_hardware)
comparison_magnitude = np.abs(comparison_inner_prod)

# Display the results
print("Encoded Quantum State (Direct Method):", encoded_state_direct)
print("Encoded Quantum State (Hardware-Simulated):", encoded_state_hardware)
print("Inner Product Magnitude Between Methods:", comparison_magnitude)

# Check if they are approximately the same
if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states.")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.31976474+0.31976474j 0.31976474+0.31976474j 0.15988237+0.47964711j
 0.06395295+0.57557653j]
Encoded Quantum State (Hardware-Simulated): [0.12020148-0.19449009j 0.82401614+0.34131866j 0.37972203+0.08964021j
 0.        +0.j        ]
Inner Product Magnitude Between Methods: 0.6193332974241113
The two methods produce different quantum states.


In [38]:
import numpy as np

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Group pairs of reals into complex numbers and normalize.
    """
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (of length 2^n) from |0...0>
    by computing, at each binary tree node, the rotation angle (theta) and relative
    phase (phi) that split the probability amplitude between the left (|0>) and right (|1>) branches.
    """
    n = int(np.log2(len(psi)))
    if n == 0:
        # Base case: one amplitude.
        return psi

    half = len(psi) // 2
    # Compute the norms (i.e. the probability masses) of the two branches.
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])

    # The rotation on the current qubit should prepare a state whose amplitudes are:
    # cos(theta/2) = norm_left and sin(theta/2) = norm_right (up to a phase).
    theta = 2 * np.arctan2(norm_right, norm_left)

    # Determine a relative phase between branches.
    # (We choose the phase difference between the first elements of each branch.)
    if norm_left > 0 and norm_right > 0:
        phi = np.angle(psi[half]) - np.angle(psi[0])
    else:
        phi = 0.0

    # Build the single-qubit unitary U = R_z(phi) R_y(theta)
    U = rotation_matrix_z(phi) @ rotation_matrix_y(theta)
    # Prepare the qubit state: U|0> = [cos(theta/2), e^(i phi)*sin(theta/2)]
    qubit_state = U @ np.array([1, 0], dtype=complex)

    # Recursively prepare the remainder of the state in each branch.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)

    # The overall state is the “controlled” state:
    # branch 0 (qubit in |0>) is multiplied by qubit_state[0]
    # branch 1 (qubit in |1>) is multiplied by qubit_state[1]
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using only unitary operations (as on real hardware)
    by preparing the state recursively.
    """
    # Convert real vector into complex amplitudes.
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

# Example vector for encoding (must have even length; here it encodes a 4-dimensional state)
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Encode state using both methods.
encoded_state_direct = encode_to_qubit(vector)
encoded_state_hardware = amplitude_encoding_real_hardware(vector)

# Verify if both methods result in (roughly) the same state.
comparison_inner_prod = inner_product(encoded_state_direct, encoded_state_hardware)
comparison_magnitude = np.abs(comparison_inner_prod)

print("Encoded Quantum State (Direct Method):", encoded_state_direct)
print("Encoded Quantum State (Hardware-Simulated):", encoded_state_hardware)
print("Inner Product Magnitude Between Methods:", comparison_magnitude)

if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states.")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.31976474+0.31976474j 0.31976474+0.31976474j 0.15988237+0.47964711j
 0.06395295+0.57557653j]
Encoded Quantum State (Hardware-Simulated): [ 0.38467759+0.23774383j  0.38467759+0.23774383j  0.0982017 +0.49596386j
 -0.13017201+0.56429918j]
Inner Product Magnitude Between Methods: 0.9693689725158599
The two methods produce different quantum states.


In [39]:
import numpy as np

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Group pairs of reals into complex numbers and normalize.
    """
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (of length 2^n) from |0...0>
    by computing, at each binary tree node, the rotation angle and relative phase.
    
    At each level, we adjust the overall phase so that the first amplitude is real
    and nonnegative. This avoids phase drifts between branches.
    """
    # Adjust global phase of the current state so that psi[0] is real and nonnegative.
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
        
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi

    half = len(psi) // 2
    # Compute the norms of the left and right subtrees.
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    # For a normalized state, norm_left^2 + norm_right^2 = 1.
    # The rotation angle is then:
    theta = 2 * np.arctan2(norm_right, norm_left)
    
    # Since we fixed psi[0] to be real, we set the phase difference using the first element of the right branch.
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0

    # Build the unitary rotation on the current qubit:
    U = rotation_matrix_z(phi) @ rotation_matrix_y(theta)
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    # Recursively prepare the left and right branches.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using only unitary operations
    by recursively preparing the state.
    
    We first form the target state (normalizing as needed) and fix its global phase.
    """
    # Convert real vector into complex amplitudes.
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Fix the global phase so that the first element is real and nonnegative.
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    
    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

# Example vector for encoding (even number of elements, encoding a 4-d state)
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Encode state using both methods.
encoded_state_direct = encode_to_qubit(vector)
encoded_state_hardware = amplitude_encoding_real_hardware(vector)

# Verify if both methods result in (roughly) the same state (up to a global phase).
comparison_inner_prod = inner_product(encoded_state_direct, encoded_state_hardware)
comparison_magnitude = np.abs(comparison_inner_prod)

print("Encoded Quantum State (Direct Method):", encoded_state_direct)
print("Encoded Quantum State (Hardware-Simulated):", encoded_state_hardware)
print("Inner Product Magnitude Between Methods:", comparison_magnitude)

if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states.")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.31976474+0.31976474j 0.31976474+0.31976474j 0.15988237+0.47964711j
 0.06395295+0.57557653j]
Encoded Quantum State (Hardware-Simulated): [0.44011841-0.10389786j 0.44011841-0.10389786j 0.50156673+0.06367523j
 0.54647269+0.19169227j]
Inner Product Magnitude Between Methods: 0.9986539588256725
The two methods produce different quantum states.


In [2]:
import numpy as np

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Groups pairs of reals into complex numbers and normalizes.
    """
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (with 2^n amplitudes) from |0...0>
    by splitting the state into left and right halves. At each step,
    we compute the rotation angle theta and relative phase phi so that
      cos(theta/2) = norm(left branch)
      sin(theta/2) = norm(right branch)
    and we force the left branch amplitude to be real and nonnegative.
    
    The unitary on the current qubit is defined as:
      U' = exp(i*phi/2)*R_z(phi)*R_y(theta)
    so that U'|0> = [cos(theta/2), sin(theta/2)*exp(i*phi)].
    """
    # Force global phase so that psi[0] is real and nonnegative.
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
    
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi

    half = len(psi) // 2
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    # Compute rotation angle: cos(theta/2)=norm_left, sin(theta/2)=norm_right.
    theta = 2 * np.arctan2(norm_right, norm_left)
    
    # Set the relative phase from the right branch.
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0

    # Build the corrected single-qubit unitary.
    U = np.exp(1j * phi/2) * (rotation_matrix_z(phi) @ rotation_matrix_y(theta))
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    # Recursively prepare the left and right branches.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using only unitary operations
    (as on real quantum hardware) by recursively preparing the state.
    """
    # Convert the real vector into complex amplitudes.
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Force global phase so that the first amplitude is real and nonnegative.
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    
    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

# Example vector for encoding (must have an even number of elements).
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Encode the state using both methods.
encoded_state_direct = encode_to_qubit(vector)
encoded_state_hardware = amplitude_encoding_real_hardware(vector)

# Optionally, force the same global phase on both states.
def fix_global_phase(state):
    if state[0] != 0:
        return state * np.exp(-1j * np.angle(state[0]))
    return state

encoded_state_direct = fix_global_phase(encoded_state_direct)
encoded_state_hardware = fix_global_phase(encoded_state_hardware)

# Compute the inner product between the two states.
comparison_inner_prod = inner_product(encoded_state_direct, encoded_state_hardware)
comparison_magnitude = np.abs(comparison_inner_prod)

print("Encoded Quantum State (Direct Method):", encoded_state_direct)
print("Encoded Quantum State (Hardware-Simulated):", encoded_state_hardware)
print("Inner Product Magnitude Between Methods:", comparison_magnitude)

if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states (up to a global phase).")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.45221563+3.89009787e-17j 0.45221563+3.89009787e-17j
 0.45221563+2.26107816e-01j 0.45221563+3.61772505e-01j]
Encoded Quantum State (Hardware-Simulated): [0.45221563+6.53691246e-34j 0.45221563-7.74608296e-34j
 0.45221563+2.26107816e-01j 0.45221563+3.61772505e-01j]
Inner Product Magnitude Between Methods: 1.0
The two methods produce equivalent quantum states (up to a global phase).


In [49]:
import numpy as np
from scipy.linalg import expm

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Groups pairs of reals into complex numbers and normalizes.
    """
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def adiabatic_rotation_gate(theta, phi, steps=1000, T=20.0):
    """
    Adiabatically implements a qubit rotation that takes |0⟩ to
      |ψ_target⟩ = cos(θ/2)|0⟩ + e^(i φ) sin(θ/2)|1⟩.
    
    We use a smooth schedule:
      s(t) = sin²( (π t)/(2T) )
    so that at time t the instantaneous target is:
      |ψ(s)⟩ = cos(sθ/2)|0⟩ + e^(i sφ) sin(sθ/2)|1⟩.
    
    The Hamiltonian is taken as:
      H(s) = I - |ψ(s)⟩⟨ψ(s)|
    and the evolution is applied in small time steps.
    """
    dt = T / steps
    U = np.eye(2, dtype=complex)
    for i in range(steps):
        t = (i + 1) * dt
        # Use a smooth schedule for the ramp.
        s = np.sin(np.pi * t / (2 * T))**2
        current_theta = s * theta
        current_phi = s * phi
        psi_inst = np.array([
            np.cos(current_theta/2),
            np.exp(1j * current_phi) * np.sin(current_theta/2)
        ])
        H_inst = np.eye(2, dtype=complex) - np.outer(psi_inst, np.conjugate(psi_inst))
        U_step = expm(-1j * H_inst * dt)
        U = U_step @ U
    return U

def recursive_state_preparation_adiabatic(psi, steps=1000, T=20.0):
    """
    Recursively prepares a state |psi⟩ (with 2^n amplitudes) from |0...0⟩
    using adiabatic qubit rotations.
    
    At each recursion level:
      1. Fix the global phase so that psi[0] is real and nonnegative.
      2. Split psi into left/right halves and compute:
             θ = 2 arctan(norm(right)/norm(left))
         and
             φ = phase(first element of right half).
      3. Use adiabatic_rotation_gate to steer the qubit subspace.
      4. Recurse on the normalized left/right branches.
    """
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
    
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi
    
    half = len(psi) // 2
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    theta = 2 * np.arctan2(norm_right, norm_left)
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0
    
    U = adiabatic_rotation_gate(theta, phi, steps=steps, T=T)
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation_adiabatic(psi_left, steps=steps, T=T)
    else:
        left_state = np.zeros(half, dtype=complex)
    
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation_adiabatic(psi_right, steps=steps, T=T)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware_adiabatic(vector, steps=1000, T=20.0):
    """
    Amplitude encoding via adiabatic evolution using qubit rotations.
    
    The real vector (with even number of entries) is grouped into complex amplitudes,
    normalized, and then the state is prepared recursively using adiabatic rotation gates.
    """
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    return recursive_state_preparation_adiabatic(target_state, steps=steps, T=T)

def fix_global_phase(state):
    """Force the first amplitude to be real and nonnegative."""
    if state[0] != 0:
        return state * np.exp(-1j * np.angle(state[0]))
    return state

# Example vector for encoding (even number of entries).
vector = np.array([0.5, 0.5, 0.5, 0.5, 0.25, 0.75, 0.1, 0.9])

# Direct amplitude encoding.
encoded_state_direct = fix_global_phase(encode_to_qubit(vector))

# Adiabatic evolution via qubit rotations.
encoded_state_adiabatic = fix_global_phase(amplitude_encoding_real_hardware_adiabatic(vector, steps=1000, T=20.0))

# Final global phase alignment: remove any remaining phase offset.
phase_offset = np.angle(inner_product(encoded_state_direct, encoded_state_adiabatic))
encoded_state_adiabatic *= np.exp(-1j * phase_offset)

# Compare the two methods.
comparison_inner_prod = inner_product(encoded_state_direct, encoded_state_adiabatic)
comparison_magnitude = np.abs(comparison_inner_prod)

print("Encoded Quantum State (Direct Method):", encoded_state_direct)
print("Encoded Quantum State (Adiabatic Rotations):", encoded_state_adiabatic)
print("Inner Product Magnitude:", comparison_magnitude)
if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
    print("The two methods produce equivalent quantum states (up to a global phase).")
else:
    print("The two methods produce different quantum states.")


Encoded Quantum State (Direct Method): [0.45221563+3.89009787e-17j 0.45221563+3.89009787e-17j
 0.45221563+2.26107816e-01j 0.45221563+3.61772505e-01j]
Encoded Quantum State (Adiabatic Rotations): [0.43972835+0.01867416j 0.45177009+0.01058713j 0.45222539+0.21982814j
 0.47246825+0.35499787j]
Inner Product Magnitude: 0.9994437794790029
The two methods produce different quantum states.


In [10]:
import json
import os
import numpy as np
from scipy.linalg import expm

def load_embeddings():
    """Loads stored embeddings from a JSON file if it exists."""
    EMBEDDINGS_FILE = "embeddings.json"
    if os.path.exists(EMBEDDINGS_FILE):
        with open(EMBEDDINGS_FILE, "r") as f:
            return json.load(f)
    return {}

def get_embedding(word, stored_embeddings):
    """Retrieves the embedding for the given word from the cache."""
    if word in stored_embeddings:
        print(f"Loaded cached embedding for '{word}'")
        return stored_embeddings[word]
    return None

def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)

def encode_to_qubit(arr):
    """
    Direct amplitude encoding:
    Groups pairs of reals into complex numbers and normalizes.
    """
    # Ensure the vector has an even number of elements.
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

def rotation_matrix_y(theta):
    """Construct an R_y(theta) rotation matrix."""
    return np.array([
        [np.cos(theta/2), -np.sin(theta/2)],
        [np.sin(theta/2),  np.cos(theta/2)]
    ], dtype=complex)

def rotation_matrix_z(phi):
    """Construct an R_z(phi) phase shift matrix."""
    return np.array([
        [np.exp(-1j * phi/2), 0],
        [0, np.exp(1j * phi/2)]
    ], dtype=complex)

def recursive_state_preparation(psi):
    """
    Recursively prepares a state |psi> (with 2^n amplitudes) from |0...0>
    by splitting the state into left and right halves. At each step,
    we compute the rotation angle theta and relative phase phi so that
      cos(theta/2) = norm(left branch)
      sin(theta/2) = norm(right branch)
    and we force the left branch amplitude to be real and nonnegative.
    
    The unitary on the current qubit is defined as:
      U' = exp(i*phi/2)*R_z(phi)*R_y(theta)
    so that U'|0> = [cos(theta/2), sin(theta/2)*exp(i*phi)].
    """
    # Force global phase so that psi[0] is real and nonnegative.
    if psi[0] != 0:
        psi = psi * np.exp(-1j * np.angle(psi[0]))
    
    n = int(np.log2(len(psi)))
    if n == 0:
        return psi

    half = len(psi) // 2
    norm_left = np.linalg.norm(psi[:half])
    norm_right = np.linalg.norm(psi[half:])
    
    # Compute rotation angle: cos(theta/2)=norm_left, sin(theta/2)=norm_right.
    theta = 2 * np.arctan2(norm_right, norm_left)
    
    # Set the relative phase from the right branch.
    phi = np.angle(psi[half]) if (norm_left > 0 and norm_right > 0) else 0.0

    # Build the corrected single-qubit unitary.
    U = np.exp(1j * phi/2) * (rotation_matrix_z(phi) @ rotation_matrix_y(theta))
    qubit_state = U @ np.array([1, 0], dtype=complex)
    
    # Recursively prepare the left and right branches.
    if norm_left > 0:
        psi_left = psi[:half] / norm_left
        left_state = recursive_state_preparation(psi_left)
    else:
        left_state = np.zeros(half, dtype=complex)
    if norm_right > 0:
        psi_right = psi[half:] / norm_right
        right_state = recursive_state_preparation(psi_right)
    else:
        right_state = np.zeros(half, dtype=complex)
    
    return np.concatenate([qubit_state[0] * left_state,
                           qubit_state[1] * right_state])

def amplitude_encoding_real_hardware(vector):
    """
    Simulates amplitude encoding using only unitary operations
    (as on real quantum hardware) by recursively preparing the state.
    """
    # Convert the real vector into complex amplitudes.
    complex_vec = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    norm = np.linalg.norm(complex_vec)
    if norm == 0:
        raise ValueError("Vector cannot be zero.")
    target_state = complex_vec / norm

    # Force global phase so that the first amplitude is real and nonnegative.
    if target_state[0] != 0:
        target_state = target_state * np.exp(-1j * np.angle(target_state[0]))
    
    # Prepare the state recursively.
    return recursive_state_preparation(target_state)

def fix_global_phase(state):
    """Force the first amplitude to be real and nonnegative."""
    if state[0] != 0:
        return state * np.exp(-1j * np.angle(state[0]))
    return state

# --- Use the 'cat' embedding ---

stored_embeddings = load_embeddings()
cat_vec = get_embedding("cat", stored_embeddings)
if cat_vec is None:
    print("No embedding found for 'cat'")
else:
    cat_vec = np.array(cat_vec)
    
    # Direct amplitude encoding on cat_vec.
    cat_state_direct = fix_global_phase(encode_to_qubit(cat_vec))
    
    # Simulated hardware state preparation on cat_vec.
    cat_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(cat_vec))
    
    # Optionally, force the same global phase.
    phase_offset = np.angle(inner_product(cat_state_direct, cat_state_hardware))
    cat_state_hardware *= np.exp(-1j * phase_offset)
    
    # Compute the inner product.
    comparison_inner_prod = inner_product(cat_state_direct, cat_state_hardware)
    comparison_magnitude = np.abs(comparison_inner_prod)
    
    #print("Encoded Quantum State (Direct Method):", cat_state_direct)
    #print("Encoded Quantum State (Hardware-Simulated):", cat_state_hardware)
    print("Inner Product Magnitude Between Methods:", comparison_magnitude)
    
    if np.isclose(comparison_magnitude, 1.0, atol=1e-6):
        print("The two methods produce equivalent quantum states (up to a global phase).")
    else:
        print("The two methods produce different quantum states.")


Loaded cached embedding for 'cat'
Inner Product Magnitude Between Methods: 0.9999999999999996
The two methods produce equivalent quantum states (up to a global phase).


In [11]:
cat_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(cat_vec))
dog_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(dog_vec))
car_state_hardware = fix_global_phase(amplitude_encoding_real_hardware(car_vec))

cat_dog_inner_prod = inner_product(cat_state_direct, dog_state_hardware)
cat_dog_magnitude = np.abs(cat_dog_inner_prod)

cat_car_inner_prod = inner_product(cat_state_direct, car_state_hardware)
cat_car_magnitude = np.abs(cat_car_inner_prod)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod)
print("cat/car Inner product:", cat_car_inner_prod)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude)
print("cat/car Magnitude of inner product:", cat_car_magnitude)

cat/dog Inner product: (0.843619514241996-0.1824300194768404j)
cat/car Inner product: (0.8434195949134786-0.056041380110405964j)
cat/dog Magnitude of inner product: 0.8631191092869058
cat/car Magnitude of inner product: 0.8452793913072147


In [12]:
print(len(cat_vec))

1536


In [101]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

def normalize(vector):
    """Normalize a complex vector."""
    norm = np.linalg.norm(vector)
    if norm == 0:
        raise ValueError("Zero vector cannot be normalized.")
    return vector / norm

def pair_real_to_complex(vector):
    """Convert a real-valued vector into a complex-valued vector by pairing elements."""
    if len(vector) % 2 != 0:
        raise ValueError("Vector length must be even to pair into complex numbers.")
    complex_vector = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    return complex_vector

def pad_vector(vector, target_size):
    """Pad the vector with zeros to the nearest power of 2."""
    if len(vector) > target_size:
        raise ValueError("Vector is larger than the target size.")
    padded_vector = np.zeros(target_size, dtype=complex)
    padded_vector[:len(vector)] = vector  # Copy original values
    return padded_vector

def get_rotation_angles(state):
    """Compute rotation angles theta and phi from state vector."""
    norm_left = np.linalg.norm(state[:len(state)//2])
    norm_right = np.linalg.norm(state[len(state)//2:])
    
    theta = 2 * np.arctan2(norm_right, norm_left)  # Ry rotation
    phi = np.angle(state[len(state)//2]) if (norm_left > 0 and norm_right > 0) else 0.0  # Rz phase shift
    
    return theta, phi

def apply_recursive_state_preparation(qc, qubits, state, depth=0):
    """Recursively prepare the quantum state using unitary operations."""
    n = len(state)
    if n == 1:
        return  # Base case: single amplitude

    half = n // 2
    theta, phi = get_rotation_angles(state)

    # Apply Rz and Ry to the current qubit
    target_qubit = qubits[depth]
    qc.rz(phi, target_qubit)
    qc.ry(theta, target_qubit)

    # Recursively prepare left and right sub-states
    left_state = normalize(state[:half]) if np.linalg.norm(state[:half]) > 0 else np.zeros(half, dtype=complex)
    right_state = normalize(state[half:]) if np.linalg.norm(state[half:]) > 0 else np.zeros(half, dtype=complex)

    # Ensure we have enough qubits left before applying controlled operations
    if depth + 1 < len(qubits):
        qc.cx(target_qubit, qubits[depth + 1])
        apply_recursive_state_preparation(qc, qubits, left_state, depth + 1)
        apply_recursive_state_preparation(qc, qubits, right_state, depth + 1)

def prepare_amplitude_encoding(vector):
    """Create a Qiskit quantum circuit that prepares the amplitude-encoded state."""
    vector = np.array(vector)
    
    # Step 1: Convert reals to complex numbers
    complex_vector = pair_real_to_complex(vector)
    
    # Step 2: Compute the nearest power of 2
    num_qubits = int(np.ceil(np.log2(len(complex_vector))))
    target_size = 2 ** num_qubits  # Next power of 2 (1024 for 768 complex values)
    
    # Step 3: Pad and normalize the vector
    padded_vector = pad_vector(complex_vector, target_size)
    normalized_vector = normalize(padded_vector)
    
    # Step 4: Create quantum circuit
    qc = QuantumCircuit(num_qubits)
    qubits = list(range(num_qubits))

    apply_recursive_state_preparation(qc, qubits, normalized_vector)

    # Step 5: **Explicitly save the statevector (Fix for Qiskit 1+)**
    qc.save_statevector()

    return qc

# --- Example Usage ---
vector = np.random.rand(1536)  # Example 1536D real vector
qc = prepare_amplitude_encoding(vector)

# --- Simulating the state ---
# backend = Aer.get_backend('statevector_simulator')
# job = execute(qc, backend)
# result = job.result()
# state = result.get_statevector()


In [102]:
# --- Running the circuit on AerSimulator (Statevector Mode) ---
simulator = AerSimulator(method='statevector')

# Transpile for the AerSimulator
compiled_circuit = transpile(qc, simulator)

# Run the circuit and retrieve the statevector
job = simulator.run(compiled_circuit)
result = job.result()
psi_0 = result.get_statevector()

# --- Output Results ---
print("Final Quantum State (First 10 Amplitudes):", psi_0.data[:10])
print("Vector Length:", len(np.asarray(psi_0)))

Final Quantum State (First 10 Amplitudes): [ 0.07295822+0.00384661j -0.00048827-0.05106022j  0.0094723 +0.01783942j
 -0.01234115+0.00757249j  0.00564577+0.00109971j  0.00052169-0.00398609j
 -0.00808388+0.03367567j -0.0244542 -0.00428227j  0.01171497+0.00288548j
  0.00150355-0.00829737j]
Vector Length: 1024


In [103]:
import numpy as np
from scipy.linalg import logm
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

def get_hamiltonian(circuit):
    """Compute the Hamiltonian H such that U = exp(-iH) for a given quantum circuit."""
    # Get the unitary matrix representation of the circuit
    U = Operator(circuit).data  # Extract the matrix

    # Compute the matrix logarithm to get the Hamiltonian
    H = -1j * logm(U)

    return H
    
U = Operator(qc).data
print("\nSize of the U:", U.shape)

H = get_hamiltonian(qc)
print("\nSize of the H:", H.shape)

QiskitError: 'Cannot apply Operation: save_statevector'

In [77]:
from qiskit.quantum_info import Operator, SparsePauliOp

def hamiltonian_to_pauli(H):
    """Decomposes a Hamiltonian matrix into a sum of Pauli matrices."""
    pauli_op = SparsePauliOp.from_operator(H)
    return pauli_op

# Convert Hamiltonian to a sum of Pauli operators
pauli_string = hamiltonian_to_pauli(H)

#with open("hamiltonian_pauli.txt", "w") as f:
#    f.write(str(pauli_string))
#print("Hamiltonian saved to hamiltonian_pauli.txt")



# Print results
#print("Hamiltonian as a sum of Pauli matrices:\n", pauli_string)

In [105]:
import numpy as np
from qiskit.quantum_info import Statevector, SparsePauliOp

def state_to_hamiltonian(statevector, energy=1):
    """Constructs a Hamiltonian H such that H |ψ⟩ = E |ψ⟩"""
    psi = np.array(statevector)  # Ensure it's a NumPy array
    H = energy * np.outer(psi, psi.conj())  # Compute H = E |ψ⟩⟨ψ|
    return H

# Compute the Hamiltonian
H = state_to_hamiltonian(psi_0, energy=1)

# Print results
print("Hamiltonian Matrix:\n", np.round(H, 4))
print("\nSize of H:", H.shape, "->", H.nbytes, "bytes")

Hamiltonian Matrix:
 [[ 0.0053+0.j     -0.0002+0.0037j  0.0008-0.0013j ...  0.0005-0.0005j
   0.0028-0.001j  -0.0006-0.002j ]
 [-0.0002-0.0037j  0.0026+0.j     -0.0009-0.0005j ... -0.0004-0.0003j
  -0.0008-0.0019j -0.0014+0.0005j]
 [ 0.0008+0.0013j -0.0009+0.0005j  0.0004+0.j     ...  0.0002+0.j
   0.0006+0.0005j  0.0004-0.0004j]
 ...
 [ 0.0005+0.0005j -0.0004+0.0003j  0.0002-0.j     ...  0.0001+0.j
   0.0003+0.0002j  0.0001-0.0002j]
 [ 0.0028+0.001j  -0.0008+0.0019j  0.0006-0.0005j ...  0.0003-0.0002j
   0.0017-0.j      0.0001-0.0012j]
 [-0.0006+0.002j  -0.0014-0.0005j  0.0004+0.0004j ...  0.0001+0.0002j
   0.0001+0.0012j  0.0009+0.j    ]]

Size of H: (1024, 1024) -> 16777216 bytes


In [98]:
import numpy as np
from qiskit.quantum_info import Statevector

def state_to_hamiltonian(statevector, energy=1):
    """Constructs a Hamiltonian H such that H |ψ⟩ = E |ψ⟩"""
    psi = np.array(statevector)  # Convert to NumPy array
    H = energy * np.outer(psi, psi.conj())  # Compute H = E |ψ⟩⟨ψ|
    return H

# Example: Define a quantum state
#psi = Statevector.from_label("00").data  # Example: |00⟩ state

# Compute the Hamiltonian
H = state_to_hamiltonian(psi_0, energy=1)

# Print the result
print("Hamiltonian Matrix:\n", np.round(H, 4))
print("\nSize of H:", H.shape, "->", H.nbytes, "bytes")

Hamiltonian Matrix:
 [[ 0.0053+0.j     -0.0002+0.0037j  0.0008-0.0013j ...  0.0005-0.0005j
   0.0028-0.001j  -0.0006-0.002j ]
 [-0.0002-0.0037j  0.0026+0.j     -0.0009-0.0005j ... -0.0004-0.0003j
  -0.0008-0.0019j -0.0014+0.0005j]
 [ 0.0008+0.0013j -0.0009+0.0005j  0.0004+0.j     ...  0.0002+0.j
   0.0006+0.0005j  0.0004-0.0004j]
 ...
 [ 0.0005+0.0005j -0.0004+0.0003j  0.0002-0.j     ...  0.0001+0.j
   0.0003+0.0002j  0.0001-0.0002j]
 [ 0.0028+0.001j  -0.0008+0.0019j  0.0006-0.0005j ...  0.0003-0.0002j
   0.0017-0.j      0.0001-0.0012j]
 [-0.0006+0.002j  -0.0014-0.0005j  0.0004+0.0004j ...  0.0001+0.0002j
   0.0001+0.0012j  0.0009+0.j    ]]

Size of H: (1024, 1024) -> 16777216 bytes


In [84]:
import numpy as np
from qiskit.quantum_info import Statevector

def hamiltonian_to_state(H):
    """Extracts the quantum state from the Hamiltonian."""
    eigenvalues, eigenvectors = np.linalg.eigh(H)  # Diagonalize H
    idx = np.argmax(np.abs(eigenvalues))  # Find the nonzero eigenvalue
    state = eigenvectors[:, idx]  # Extract corresponding eigenvector
    return Statevector(state)  # Convert to a Qiskit statevector

# # Example: Define a state and construct its Hamiltonian
# psi_original = Statevector.from_label("00").data
# H = np.outer(psi_original, psi_original.conj())  # H = |ψ⟩⟨ψ|

# Recover the state from H
psi_reconstructed = hamiltonian_to_state(H)

# Print results
#print("Original Statevector:\n", psi_original)
print("Reconstructed Statevector:\n", psi_reconstructed)


Reconstructed Statevector:
 Statevector([ 0.0178043 +0.j        , -0.00765934-0.01031248j,
              0.03763676+0.02849254j, ..., -0.00599574+0.02037528j,
              0.03363339+0.02683141j, -0.00101295+0.02931106j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2))


In [58]:
import numpy as np
from qiskit.quantum_info import Statevector

def check_states_equal(psi1, psi2):
    """Checks if two quantum states are equal using the inner product."""
    inner_product = np.abs(np.vdot(psi1, psi2))  # Compute inner product |⟨ψ|φ⟩|
    return inner_product

# Check if they are equal
inner_product = check_states_equal(state, psi_reconstructed)

print("Inner Product:", inner_product)

# If inner_product ≈ 1, the states are identical
if np.isclose(inner_product, 1):
    print("The states are equal (up to a global phase).")
else:
    print("The states are NOT equal.")

Inner Product: 0.9999999999999996
The states are equal (up to a global phase).


In [59]:
cat_vec.size

1536

In [62]:
import sys

cat_vec_size = sys.getsizeof(cat_vec)
print(f"Size of cat_vec in bytes: {cat_vec_size}")

state_size = sys.getsizeof(state)
print(f"Size of quantum state in bytes: {cat_vec_size}")

Size of cat_vec in bytes: 12400
Size of quantum state in bytes: 12400


In [87]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import PauliEvolutionGate
import numpy as np

# Ensure strictly real-valued matrices
H_real = np.real(H).astype(float)
H_imag = np.imag(H).astype(float)

# Convert to Qiskit Operators
op_real = Operator(H_real)
op_imag = Operator(H_imag)  # This is still real-valued

# Convert Operators to SparsePauliOp (must have real coefficients)
pauli_op_real = SparsePauliOp.from_operator(op_real)
pauli_op_imag = SparsePauliOp.from_operator(op_imag)  # Will NOT throw an error now!

# Evolution time
t = 1.0

# Create evolution gate for the real part
evolution_gate_real = PauliEvolutionGate(pauli_op_real, t)

# Build circuit for real part
num_qubits = int(np.log2(H.shape[0]))
ancilla = QuantumRegister(1, "ancilla")  # Add an ancilla qubit
qc = QuantumCircuit(num_qubits + 1)  # Extra qubit for encoding the imaginary part

# Apply real evolution
qc.append(evolution_gate_real, range(num_qubits))

# Encode the imaginary part using a Pauli rotation (RX or RZ)
for qubit in range(num_qubits):
    qc.rz(-2 * t, qubit)  # Encode imaginary part using phase rotation

qc.save_statevector()

# Print decomposed version (for real hardware execution)
#print(qc.decompose().draw())


In [107]:
# --- Running the circuit on AerSimulator (Statevector Mode) ---
simulator = AerSimulator(method='statevector')

# Transpile for the AerSimulator
compiled_circuit = transpile(qc, simulator)

# Run the circuit and retrieve the statevector
job = simulator.run(compiled_circuit)
result = job.result()
psi_1 = result.get_statevector()

# --- Output Results ---
print("Final Quantum State (First 10 Amplitudes):", psi_1.data[:10])
print("Vector Length:", len(np.asarray(psi_1)))

Final Quantum State (First 10 Amplitudes): [ 9.97634825e-01-4.29897443e-03j  1.78014697e-06+5.50887941e-05j
 -3.33158122e-04-6.25132420e-04j  3.96909366e-04+7.82341905e-04j
 -1.93314798e-04-2.15804200e-04j -3.15022217e-05-5.33836413e-05j
  1.84172538e-04+4.19679788e-04j  8.01876972e-04+1.48023442e-03j
 -3.82101684e-04-7.46745376e-04j -5.35634974e-05-5.66644371e-05j]
Vector Length: 2048


In [91]:
import numpy as np

def is_hermitian(H):
    """Check if the Hamiltonian is Hermitian (H = H†)."""
    return np.allclose(H, H.conj().T)

def is_unitary(H):
    """Check if the matrix is unitary (H†H = I)."""
    identity = np.eye(H.shape[0])
    return np.allclose(H.conj().T @ H, identity)

# ✅ Check Hermitian property
print("Is H Hermitian?", is_hermitian(H))

# ✅ Check Unitary property (should usually be False for a Hamiltonian)
print("Is H Unitary?", is_unitary(H))

Is H Hermitian? True
Is H Unitary? False


In [106]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import PauliEvolutionGate
import numpy as np

# ✅ Extract real and imaginary parts
H_real = np.real(H)
H_imag = np.imag(H)

# ✅ Construct block-encoded Hamiltonian
H_block = np.block([
    [H_real, -H_imag],
    [H_imag, H_real]
])

# ✅ Convert to SparsePauliOp
op_block = SparsePauliOp.from_operator(Operator(H_block))

# Evolution parameters
t = 1.0  # total evolution time
n = 2    # number of Trotter steps

# 🔀 Adjust for block encoding
num_qubits = int(np.log2(H_block.shape[0]))
qc = QuantumCircuit(num_qubits)

# 🌀 Trotterized time evolution with block encoding
for _ in range(n):
    for pauli, coeff in zip(op_block.paulis, op_block.coeffs):
        # Ensure real coefficients for Qiskit compatibility
        coeff_real = coeff.real
        single_term = SparsePauliOp([pauli], [coeff_real])
        evo_gate = PauliEvolutionGate(single_term, time=t/n)
        qc.append(evo_gate, range(num_qubits))

# ✅ Print decomposed circuit
# print(qc.decompose().draw())
qc.save_statevector()